<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Imports</h1>

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Constants and Global Variables</h1>

In [13]:
audio = pd.read_csv('Collected Datasets/audio.csv')
text = pd.read_csv('Collected Datasets/text.csv')

EMOTIONS = ['happiness', 'neutral', 'sadness', 'anger', 'fear', ]
MAPPER = {emotion: [] for emotion in EMOTIONS}

scaler = StandardScaler()
encoder = LabelEncoder()


In [4]:
wordMap = pd.read_csv('wordMap.csv')
for emotion in wordMap.columns.to_list():    
    if emotion.lower() in EMOTIONS: MAPPER[emotion.lower()] = wordMap[emotion].to_list()
    if emotion in ['Calm', 'Boredom']: MAPPER['neutral'] += wordMap[emotion].to_list()
    if emotion in ['Excitement', 'Pride']: MAPPER['happiness'] += wordMap[emotion].to_list()
    if emotion in ['Disgust', 'Frustration', 'Contempt']: MAPPER['anger'] += wordMap[emotion].to_list()

<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Cleaning and Preprocessing</h1>

In [5]:
def _mfccStrToArray(mfccStr):
        vals = np.array(list(map(float, mfccStr.split(','))))
        return vals.reshape(13, vals.size // 13)
    
def _mfccArrayToStr(mfccArr):
        return ','.join(map(str, mfccArr.flatten()))

In [6]:
def scale(data, f="train"):
    if f == "train":
        data = scaler.fit_transform(data)
    else:
        data = scaler.transform(data)
    return data

In [7]:
def encode(data, f="train"):
    if f == "train":
        data = encoder.fit_transform(data)
    else:
        data = encoder.transform(data)
    return data

In [ ]:
def zeroPadding(data, maxLen=120):
    data['mfcc'] = data['mfcc'].apply(_mfccStrToArray)
    def padToMaxLen(mfccArr):
        if mfccArr.shape[1] < maxLen:
            pad_width = maxLen - mfccArr.shape[1]
            mfccArr = np.pad(mfccArr, ((0,0), (0, pad_width)), mode='constant')
        return mfccArr

    data['mfcc'] = data['mfcc'].apply(padToMaxLen)
    data['mfcc'] = data['mfcc'].apply(_mfccArrayToStr)
    return data

audio = zeroPadding(audio)

0
